In [6]:
# Connect to workspace
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.13.0 to work with seed


In [7]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'house-price-training-AML'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('train.csv', os.path.join(training_folder, "train.csv"))

'house-price-training-AML/train.csv'

In [8]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='house-price-training-Ridge.ipynb',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# Create an experiment
experiment_name = 'house-price-training-AML'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=False)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


{'runId': 'house-price-training-AML_1599340180_5ccc6353',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-09-05T21:09:43.596237Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'db51b3e1-132c-4f9a-8431-45427fdd7459'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'papermill_notebook_run_handler.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': ['-i',
   'house-price-training-Ridge.ipynb',
   '-o',
   './outputs/house-price-training-Ridge.output.ipynb',
   '-e',
   '{"history": true, "timeout": null}',
   '-p',
   '{"engine_name": "azureml_engine"}',
   '-n',
   '{}'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment house-price-training-AML Environment',
   'version'

In [9]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
# Register Model
from azureml.core import Model
run.register_model(model_name = 'regression_model',
                  model_path = 'outputs/house_price_Ridge_model.pkl',
                  description = 'A regression model to predict house prices',
                  model_framework = Model.Framework.SCIKITLEARN,
                  model_framework_version='0.20.3')

Model(workspace=Workspace.create(name='seed', subscription_id='347c4df8-4298-408d-b3b2-98cb005c49a0', resource_group='mlseed'), name=regression_model, id=regression_model:2, version=2, tags={}, properties={})